Ein einfaches Dashboard-Beispiel mit einem Dienstkonto (Service Account)
-------------------------------------------------------

![](https://d33wubrfki0l68.cloudfront.net/673dbafd771491a080c02c6de3fdd41b09623c90/50100/images/docs/admin/access-control-overview.svg)

Quelle: [https://kubernetes.io/docs/reference/access-authn-authz/controlling-access/](https://kubernetes.io/docs/reference/access-authn-authz/controlling-access/)

- - -

In dieser Übung legen wir ein Dienstkonto (Service Account) an.

Dazu erstellen wir eine Role, mit den Zugriffsmethoden (verbs), den Service Account und die RoleBinding.

Für weitere Informationen siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/rbac).


In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
kind: Role
apiVersion: rbac.authorization.k8s.io/v1beta1
metadata:
  namespace: default
  name: dashboard-reader-role
rules:
- apiGroups: [""]
  resources: ["pods"]
  verbs: ["get", "watch", "list"]
---  
apiVersion: v1
kind: ServiceAccount
metadata:
  name: dashboard-pod
---
kind: RoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  name: dashboard-reader-rolebinding
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io  
  kind: Role
  name: dashboard-reader-role
subjects:
- kind: ServiceAccount
  name: dashboard-pod
  namespace: default  
%EOF%

- - -
Für den Zugriff auf das Dashboard brauchen wir den Token des Service Accounts.


In [ ]:
! kubectl describe secret $(kubectl get secret | grep dashboard-pod-token | awk '{print $1}')

Diesen Token können wir verwenden um uns in Dashboard einzuloggen.

http://localhost:8001/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy

- - -
Aufräumen

In [ ]:
! kubectl delete Role/dashboard-reader-role
! kubectl delete RoleBinding/dashboard-reader-rolebinding
! kubectl delete ServiceAccount/dashboard-pod